In [1]:
# clean imorts

import sys
sys.path.append('../..')
import os
from tqdm import tqdm
from config import client
from pymongo import MongoClient
import pandas as pd

import matplotlib.pyplot as plt
import math
import numpy as np
#import json 
#from pandas.io.json import json_normalize #package for flattening json in pandas df

import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [2]:
#client = MongoClient(client)
db = client['osu_random_db']

## Import data

In [3]:
sample_id = list(db.week_9_sample_users.find( {}, {"_id":0, "users": 1}))[0]["users"]

In [26]:
# Turn these 2 cells into python modules in mlpp/data_modeling
user_info = {}
for user in sample_id:
    user_info[user] = list(db.osu_scores_high.find( {"user_id": user, "enabled_mods": 0}, {"_id": 0, "beatmap_id": 1, "count50": 1, "count100": 1, "count300": 1, "countmiss": 1, "countgeki": 1, "countkatu": 1, "perfect": 1}))

In [4]:
user_info_with_beatmap_features = user_info
df = pd.DataFrame()
for user in sample_id:
    for bm in user_info_with_beatmap_features[user]:
        bm.update(list(db.osu_beatmaps.find( {"_id": bm["beatmap_id"]}, {"_id": 0, "countNormal": 1, "countSlider": 1, "countSpinner": 1, "countTotal": 1, "diff_approach": 1, "diff_drain": 1, "diff_overall": 1, "diff_size": 1, "diffcultyrating": 1, "hit_length": 1}))[0])
        bm.update(list(db.osu_scores_high.find( {"user_id": user, "beatmap_id": bm["beatmap_id"], "enabled_mods": 0}, {"_id": 0, "mlpp.est_user_pp": 1}))[0]["mlpp"])
    df = df.append(pd.DataFrame(user_info_with_beatmap_features[user]))

NameError: name 'user_info' is not defined

In [34]:
df

,beatmap_id,count50,count100,count300,countmiss,countgeki,countkatu,perfect,countNormal,countSlider,countSpinner,countTotal,diff_approach,diff_drain,diff_overall,diff_size,hit_length,est_user_pp
0,2037,2,76,133,5,4,20,0,155,60,1,278,5.0,4.0,5.0,5.0,155,114.756841
1,2409,6,24,70,5,14,11,0,79,24,2,133,5.0,5.0,5.0,5.0,64,132.594611
2,2607,2,7,185,1,36,5,0,158,37,0,232,3.0,2.0,3.0,3.0,209,63.021358
3,3670,2,19,230,11,63,11,0,249,12,1,276,4.0,3.0,4.0,5.0,145,61.159836
4,3851,5,41,87,9,38,19,0,92,44,6,198,5.0,5.0,5.0,6.0,129,57.935610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,1601789,7,21,141,1,35,15,0,48,120,2,294,6.0,3.0,4.0,3.0,84,47.473359
3,1603927,6,45,177,5,34,19,0,78,153,2,390,7.7,4.7,6.6,4.0,92,61.380520
4,1655999,0,12,156,0,26,8,0,70,98,0,266,5.0,4.0,4.0,3.0,77,44.234345
5,1919496,3,14,168,0,46,9,0,82,102,1,289,5.5,4.0,4.0,3.0,91,7.999520


In [35]:
df.to_csv("xgboostDf.csv") 

In [30]:
# df = pd.read_csv("xgboostDf.csv")

In [36]:
df.drop('beatmap_id', axis = 1, inplace = True)

In [37]:
df

,count50,count100,count300,countmiss,countgeki,countkatu,perfect,countNormal,countSlider,countSpinner,countTotal,diff_approach,diff_drain,diff_overall,diff_size,hit_length,est_user_pp
0,2,76,133,5,4,20,0,155,60,1,278,5.0,4.0,5.0,5.0,155,114.756841
1,6,24,70,5,14,11,0,79,24,2,133,5.0,5.0,5.0,5.0,64,132.594611
2,2,7,185,1,36,5,0,158,37,0,232,3.0,2.0,3.0,3.0,209,63.021358
3,2,19,230,11,63,11,0,249,12,1,276,4.0,3.0,4.0,5.0,145,61.159836
4,5,41,87,9,38,19,0,92,44,6,198,5.0,5.0,5.0,6.0,129,57.935610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,7,21,141,1,35,15,0,48,120,2,294,6.0,3.0,4.0,3.0,84,47.473359
3,6,45,177,5,34,19,0,78,153,2,390,7.7,4.7,6.6,4.0,92,61.380520
4,0,12,156,0,26,8,0,70,98,0,266,5.0,4.0,4.0,3.0,77,44.234345
5,3,14,168,0,46,9,0,82,102,1,289,5.5,4.0,4.0,3.0,91,7.999520


In [42]:
x, y = df.iloc[:,:-1],df.iloc[:,-1]
df_dmatrix = xgb.DMatrix(data=x,label=y)

In [49]:
params = {"objective":"reg:linear",'max_depth': 5}

cv_results = xgb.cv(dtrain=df_dmatrix, params=params, nfold=3, metrics="rmse", as_pandas=True, seed=123)

[21:54:42] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:54:42] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:54:42] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


In [50]:
cv_results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,79.337611,0.075617,79.365290,0.219152
1,65.736575,0.078841,65.804962,0.193493
2,57.727983,0.096413,57.828655,0.167277
3,53.239714,0.095500,53.381947,0.170728
4,50.734977,0.066506,50.913328,0.184590
